<a href="https://colab.research.google.com/github/ntua-unit-of-control-and-informatics/jaqpot-google-collab-examples/blob/main/Scikit-learn-models/evaluate-a-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluate a Model

In this example, we will demonstrate how to evaluate the robustness of a model using `jaqpotpy`. We will use a RandomForestRegressor model and perform various evaluations including cross-validation, external evaluation, and a randomization test.

In [54]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from jaqpotpy.models import SklearnModel
from jaqpotpy.datasets import JaqpotpyDataset
from jaqpotpy.descriptors import RDKitDescriptors

We start by creating a sample dataset with molecular structures represented as SMILES strings, along with temperature and activity values.

In [55]:
# Create sample data
data = pd.DataFrame(
    {
        "smiles": ["CC", "CCO", "CCC", "CCCl",
        "CCBr", "COC", "CCOCC", "CCCO",
        "CCCC", "CCCCCC",
    ],

        "temperature": np.random.randint(20, 37, size=10),
        
        "activity": [80, 81, 81, 84, 83.5,
        83, 89, 90, 91, 97,
    ],
    }
)

Next, we prepare the dataset for training using `JaqpotpyDataset` and `RDKitDescriptors` for feature extraction.

In [56]:
featurizer = RDKitDescriptors()

# Prepare the dataset for training with Jaqpotpy
train_dataset = JaqpotpyDataset(
    df=data,
    x_cols=["temperature"],
    y_cols=["activity"],
    smiles_cols=["smiles"],
    task="regression",
    featurizer=featurizer,
)

We then initialize a RandomForestRegressor model and wrap it with `SklearnModel` from `jaqpotpy`. The model is trained on the prepared dataset.

In [57]:
model = RandomForestRegressor(random_state=42)
jaqpot_model = SklearnModel(dataset=train_dataset, model=model)
jaqpot_model.random_seed = 1231
jaqpot_model.fit()

Goodness-of-fit metrics on training set:
{'r2': 0.9511043533351436, 'mae': 0.8829999999999985, 'rmse': 1.1621617787554346}


To estimate the model's performance, we perform cross-validation on the training data.

In [58]:
# Perform cross-validation on the training data
jaqpot_model.cross_validate(train_dataset, n_splits=5)

{'r2': 0.5357613775291498, 'mae': 2.280000000000001, 'rmse': 2.710691494968137}

We define a test dataset for external evaluation and prepare it using `JaqpotpyDataset`.

In [59]:
# Define test data for external evaluation
X_test = pd.DataFrame(
    {
        "smiles": ["CCCOC", "CO"],
        "temperature": [27.0, 22.0],
        "activity": [89.0, 86.0],
    }
)

# Prepare the test dataset with Jaqpotpy
test_dataset = JaqpotpyDataset(
    df=X_test,
    smiles_cols="smiles",
    x_cols=["temperature"],
    y_cols=["activity"],
    task="regression",
    featurizer=featurizer,
)

We evaluate the model on the test dataset to assess its performance on new/unseen data.

In [60]:
# Evaluate the model on the test dataset
jaqpot_model.evaluate(test_dataset)

predictions = jaqpot_model.predict(test_dataset)
print(predictions)

[89.205 81.58 ]


Finally, we conduct a randomization test to assess the model's robustness against randomization of target labels.

In [61]:
# Conducts a randomization test to assess the model's robustness
jaqpot_model.randomization_test(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    n_iters=10,
)

{'iteration_0': {'Train': {'r2': 0.798810842610191,
   'mae': 1.8244999999999991,
   'rmse': 2.3574027021279162},
  'Test': {'r2': -2.2556722222222225,
   'mae': 2.7025000000000006,
   'rmse': 2.706522214946702}},
 'iteration_1': {'Train': {'r2': 0.8782363109783689,
   'mae': 1.4065000000000025,
   'rmse': 1.8339622406145686},
  'Test': {'r2': -12.070577777777752,
   'mae': 5.419999999999995,
   'rmse': 5.422988106201225}},
 'iteration_2': {'Train': {'r2': 0.8840678794461037,
   'mae': 1.635000000000001,
   'rmse': 1.7895069153261192},
  'Test': {'r2': -2.0272055555555486,
   'mae': 2.112499999999997,
   'rmse': 2.6098299753049017}},
 'iteration_3': {'Train': {'r2': 0.8251997465834012,
   'mae': 1.7449999999999988,
   'rmse': 2.1973666057351466},
  'Test': {'r2': -0.3593111111111116,
   'mae': 1.6649999999999991,
   'rmse': 1.7488424743240887}},
 'iteration_4': {'Train': {'r2': 0.7538583582224638,
   'mae': 2.1194999999999995,
   'rmse': 2.6074983221471104},
  'Test': {'r2': -8.6327555